In [1]:
from edc import print_info
print_info("geodb-sharing")


***Notebook Title***  
GeoDB: Sharing Data

***Notebook Description***  
Euro Data Cube GeoDB: Sharing Data


***Notebook Dependencies***  
This notebook requires an active subscription to:
* EDC GeoDB


In [2]:
from edc import setup_environment_variables
setup_environment_variables()

API credentials have automatically been injected for your active subscriptions.  
The following environment variables are now available:
* `GEODB_API_SERVER_PORT`, `GEODB_API_SERVER_URL`, `GEODB_AUTH_AUD`, `GEODB_AUTH_CLIENT_ID`, `GEODB_AUTH_CLIENT_SECRET`, `GEODB_AUTH_DOMAIN`

The following additional environment variables have been loaded from `~/custom.env`:
* `AWS_BUCKET`
* `DAPA_URL`
* `DB_HOST`, `DB_NAME`, `DB_PASSWORD`, `DB_USER`
* `OGC_EDC_URL`
* `REFERENCE_DATA`


In [3]:
from edc import check_compatibility
check_compatibility("user-0.21.2")

## Sharing Data

In [4]:
from xcube_geodb.core.geodb import GeoDBClient

In [5]:
geodb = GeoDBClient()
geodb.whoami

'geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04'

In [6]:
ds = geodb.get_my_collections()

if ds[(ds.database == geodb.whoami) & (ds.table_name == 'land_use')].table_name.count() == 0:
    # Have a look at fiona feature schema
    collections = {
            "land_use": 
            {
                "crs": 3794,
                "properties": 
                {
                    "RABA_PID": "float", 
                    "RABA_ID": "float", 
                    "D_OD": "date"
                }
            }
        }


    geodb.create_collections(collections)
    import geopandas
    import os
    gdf = geopandas.read_file(os.path.expanduser("~/.shared/notebooks/getting-started/data/sample/land_use.shp"))
    geodb.insert_into_collection('land_use', gdf)

### Publish a Collection to the World

In [7]:
geodb.list_my_grants()

,database,table_name,grantee,privileges
0,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,PUBLIC,SELECT


In [8]:
geodb.publish_collection("land_use")

Access granted on land_use to public.

### Accessing Collection as a different User

Please be aware that you cannot change the user easily within a Notebook. Here, we changed the credentials in the background. The following cells will not run and are for demonstration only.

In [9]:
geodb = GeoDBClient()
geodb.whoami

'geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04'

In [10]:
geodb.get_collection('land_use', database='geodb_admin')

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od
0,1,2020-09-25T08:53:24.471636+00:00,None,"POLYGON ((453952.629 91124.177, 453952.696 911...",4770326,1410,2019-03-26
1,2,2020-09-25T08:53:24.471636+00:00,None,"POLYGON ((453810.376 91150.199, 453812.552 911...",4770325,1300,2019-03-26
2,3,2020-09-25T08:53:24.471636+00:00,None,"POLYGON ((456099.635 97696.070, 456112.810 976...",2305689,7000,2019-02-25
3,4,2020-09-25T08:53:24.471636+00:00,None,"POLYGON ((455929.405 97963.785, 455933.284 979...",2305596,1100,2019-02-25
4,5,2020-09-25T08:53:24.471636+00:00,None,"POLYGON ((461561.512 96119.256, 461632.114 960...",2310160,1100,2019-03-11
...,...,...,...,...,...,...,...
19649,19650,2020-09-29T08:59:47.435084+00:00,None,"POLYGON ((460637.334 96865.891, 460647.927 969...",6253989,1600,2019-03-08
19650,19651,2020-09-29T08:59:47.435084+00:00,None,"POLYGON ((459467.868 96839.686, 459467.770 968...",6252044,1600,2019-03-26
19651,19652,2020-09-29T08:59:47.435084+00:00,None,"POLYGON ((459488.998 94066.248, 459498.145 940...",6245985,2000,2019-04-08
19652,19653,2020-09-29T08:59:47.435084+00:00,None,"POLYGON ((459676.680 94000.000, 459672.469 939...",6245986,2000,2019-02-20


### Revoke access

In [11]:
geodb.list_my_grants()

,database,table_name,grantee,privileges
0,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,PUBLIC,SELECT


In [12]:
geodb.unpublish_collection("land_use")

Error: {"hint":null,"details":null,"code":"42P01","message":"relation \"\"geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04_land_use_id_seq\"\" does not exist"}

In [13]:
geodb.list_my_grants()

,database,table_name,grantee,privileges
0,geodb_418dfeac-15f0-4606-9edb-fd9eb722bf04,land_use,PUBLIC,SELECT


In [14]:
gdf = geodb.get_collection_by_bbox(collection="land_use", database="geodb_admin", bbox=(452750.0, 88909.549, 464000.0, 102486.299),
                comparison_mode="contains", bbox_crs=3794, limit=2, offset=10)
gdf

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od
0,11,2020-09-25T08:53:24.471636+00:00,None,"POLYGON ((460137.998 95628.898, 460111.001 956...",5983161,1100,2019-03-11
1,12,2020-09-25T08:53:24.471636+00:00,None,"POLYGON ((453673.609 91328.224, 453678.929 913...",5983074,1600,2019-03-26
